# Version 5

Pour réponse à la problématique d'épuiser tout le budget avant la fin de la journée, il faut pouvoir garantir un achat massif des br si jamais celles-ci venaient à se ralentir (pour être sur d'en avoir assez avant la fin de la journée pour épuiser tout el budget).

La première étape consiste donc à remodifier légèrement le cadre de la simulation en prenant en compte des effets d'affluence qui permettraient de faire varier la fréquence des br en plus du nb de panneaux dispo. Par exemple on pourrait imaginer avoir des creux de 3h en début, milieu ou fin de journée et ainsi simuler le ralentissement dans l'arrivée des bid requests. 

Une fois cela réalisé, on pourrait modifier le calcul de $b_t$ en prenant en compte justement les effets de vitesse et d'accélération. On pourrait à juste titre prendre en compte la vitesse de $b_t$ dans son calcul puisque ce dernier est directement lié à la vitesse d'arrivée des br (plus la vitesse d'arrivée des br est faible, plus la différence $b_{t+1} - b_t$ est grande.

Rappelons qu'on à la formule suivante : 
$$b_{t+1} = \Bigr(B - \sum_{s=1}^{t} S(s)\Bigl) \dfrac{1}{T-t}$$

Si nous prenons un temps continu, la vitesse de $b_t$ est mesuré par $v_t = \dfrac{b_{t+1} - b_t}{\partial b_t}$. Or étant en temps discret, au niveau d'une seconde le calcul est simplement le suivant : $v_t = b_{t+1} - b_t$.

De la même manière, l'accélération étant la dérivée seconde, on peut ici déterminer : $a_t = v_{t+1}-v_t$

L'idée est ensuite la suivante : on pourrait imaginer regarder une moyenne mobile de la vitesse sur un certain laps de temps (10min, 30min, 1h?) et utiliser cette moyenne pour la faire intervenir dans le calcul de $b_t$ directement. On aurait donc une nouvelle formule du type : 
$$b_{t+1} = \Bigr(B - \sum_{s=1}^{t} S(s)\Bigl) \dfrac{1+\alpha \overline{v_t}}{T-t}$$
où finalement $\overline{v_t}$ serait une moyenne mobile de $v_t$ et $\alpha$ un paramètre que l'on pourrait fixer en fonction des simulations. A termes, on pourrait même imaginer avoir quelque chose de la même forme mais où le $\alpha$ dépendrait de l'accélération tel que : $\alpha = \beta \times \overline{a_t}$ où là encore on pourrait fixer $\beta$.

### Reprenons le cadre de simulation pour implémenter des éventuels ralentissements

In [1]:
import numpy as np
import random 
import simpy
import time
import csv
from collections import namedtuple
from datetime import datetime
import pandas as pd

In [2]:
def panneaux_dispo(nb_heures, total_panneaux):
    """Fonction qui génère un nombre de panneaux dispo parmi 1000 panneaux pour chaque heure de la journée"""
    prop = np.random.normal(loc=0.8, scale=0.2, size=nb_heures)
    pann_dispo = list(map(lambda x: int(total_panneaux*x if x<=1 else total_panneaux), prop))
    return pann_dispo

In [3]:
def lambda_br(pann_dispo):
    """Fonction qui génère un lambda de la loi de poisson. Plus le nombre de panneaux est grand, plus lambda est faible"""
    lam = list(map(lambda x: (1000-x)/50 if x<950 else 1, pann_dispo))
    return lam

In [4]:
def imps():
    """Fonction qui génère un nombre d'impressions devant un panneau"""
    lam = int(np.random.normal(loc=4, scale=2, size=1))
    if lam < 1:
        lam = 1
    nb_imp = np.random.poisson(lam)
    return nb_imp

In [5]:
def delai(lam):
    """Fonction qui génère un nombre de secondes avant la prochaine br grâce à une loi de poisson de paramètre lambda"""
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (1% de chance d'en avoir un)
    if not random.random() < 0.99:
        secondes = np.random.poisson(lam + 1000)
    return secondes

In [6]:
def total_sec(nb_jours):
    """Fonction pour générer le nombre de secondes contenu dans X jours"""
    tot = nb_jours * 86400
    return tot

In [7]:
def sauvegarde(liste, nom_de_fichier):
    """Fonction pour sauvegarder un named tuple dans un format CSV"""
    with open(nom_de_fichier, "w", encoding="utf8") as fichier:
        #Recuperer le nom des colonnes pour la première ligne
        premier, *_ = liste
        ecrivain = csv.DictWriter(fichier, premier._fields)
        ecrivain.writeheader()
        for br in liste:
            # On ecrit chaque ligne comme un ordered dict
            ecrivain.writerow(br._asdict())

In [8]:
def open_rtb_pacing(env, P, B, timestampnow, nb_jours, nb_hours_per_day, bidrequests, data):
    """Fonction pour générer la simulation"""
    setup = True
    identifiant = 0
    cible = B/(nb_hours_per_day*3600)
    current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
    while True:
        # Déterminer dans quel état on est actuellement
        if current_hour >= 6 and current_hour < 20:
            if setup:
                setup = False
                dispo = panneaux_dispo(nb_hours_per_day, 1000)
                lam_secondes = lambda_br(dispo)
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
                i = 0
                S = list()
                end_day = env.now + (nb_hours_per_day*3600) - 1200 #On fixe la fin de la journée à 19h40h pour être plus restrictif
            
            # Génération d'une br
            identifiant += 1
            rt = (timestampnow+total_sec(nb_jours))-env.now
            
            # Déterminer b_t
            try:
                bt = (B - sum(S))/(end_day - env.now)
                
            except ZeroDivisionError:
                bt = 1
            #on force l'achat
            if bt < 0:
                bt=1
            
            #Timestamp de la br
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")

            # Nombre d'impressions
            nb_imp = imps()
            Prix = P * nb_imp
            
            #Achat ou non 
            if bt >= cible and (sum(S)+Prix) <= B:
                S.append(Prix)
                achat = 1
            else:
                achat = 0

            # Stocker les résultats dans le namedtuple puis ajout à une liste
            resultats = bidrequests(
                identifiant = identifiant,
                timestamp = env.now,
                timestamp_string = time,
                nombre_impressions = nb_imp,
                prix_impression = P,
                prix_total = Prix,
                achat = achat,
                budget_depense = sum(S),
                bt = bt,
                cible = cible
            )
            data.append(resultats)

            # Temps avant la prochaine BR
            # On passe par un try except au cas ou il y a un changement d'heure inattendu sur la fin de période
            try:
                time_before_next = delai(lam_secondes[i])
            except IndexError:
                time_before_next = delai(lam_secondes[i-1])


            # Fin de la simulation
            if rt < time_before_next:
                print(f"Fin de la simulation à {time} !")

            yield env.timeout(time_before_next)
            
            # Detecter quelle heure est-t-il
            if datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour != current_hour:
                i += 1 
                current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour

        else:
            setup = True
            current_hour = datetime.strptime(datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S"), '%m-%d-%Y %H:%M:%S').hour
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")
            rt = (timestampnow+total_sec(nb_jours))-env.now
            if rt <= 1:
                print(f"Fin de la simulation à {time} !")
            yield env.timeout(1) 